***the objectif behind this code is to label the data (SVM Model output: macro classification ("générique" & "non générique" )) in order to classify the keywords into two subgategories: model & brand("marque")***

In [54]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from unidecode import unidecode

In [55]:
corpus = pd.read_csv(r'C:/Users/ASUS/Desktop/stage pfe/model output/general_cat_pred_svm.csv')
corpus

,Keyword,prediction
0,accessoires auto tunisie,générique
1,accessoire voiture tunisie,générique
2,maison volkswagen tunisie pièce de rechange,non générique
3,enjoliveur tunisie,générique
4,accessoire voiture tunisie prix,générique
...,...,...
5751,location utilitaire tunis,générique
5752,circuit 4x4 tunisie marmara,générique
5753,excursion 4x4 hammamet,générique
5754,raid 4x4 en tunisie,générique


In [56]:
df = pd.DataFrame()
# Filter rows where 'prediction' equals 'non générique'
filtered_rows = corpus[corpus["prediction"] == "non générique"]

# Concatenate the filtered rows to df
df = pd.concat([df, filtered_rows], ignore_index=True)



In [57]:
df

,Keyword,prediction
0,maison volkswagen tunisie pièce de rechange,non générique
1,prix pièce de rechange kia rio tunisie,non générique
2,accessoires isuzu d max tunisie,non générique
3,pièce de rechange volkswagen tunisie,non générique
4,pièce de rechange volkswagen sfax,non générique
...,...,...
3207,c3 aircross tunisie,non générique
3208,prix peugeot 2008 en tunisie,non générique
3209,citroen c3 aircross tunisie,non générique
3210,prix hyundai tucson en tunisie,non générique


In [58]:
df.shape

(3212, 2)

**STEP 1: DATA COLLECTION**

importing a dataset from Kaggle to use it in the labelisation step

In [59]:
labelDf = pd.read_excel(r"C:/Users/ASUS/Desktop/stage pfe/collected data/CarDB.xlsx", sheet_name = "CarModels")

In [60]:
labelDf

,ID,BRAND,MODEL,YEAR_,FUEL,SHIFTTYPE,MOTORVOLUME,CASETYPE,VARIANT
0,1,Audi,100,1993,Gasoline/LPG,Automatic transmission,1301-1600 cc,Sedan,2.0
1,2,Audi,100,2006,Gasoline/LPG,Automatic transmission,2001-2500 cc,Sedan,2.0
2,3,Audi,80,1982,Diesel,Manual Gear,-,Sedan,Audi 80 Series 1.6 D
3,4,Audi,80,1991,Diesel,Manual Gear,1301-1600 cc,Sedan,Audi 80 Series 1.6 D
4,5,Audi,A1,2011,Gasoline,Semi-Automatic Gear,1301-1600 cc,Hatchback 3 Doors,Audi A1 1.4 TFSI Ambition
...,...,...,...,...,...,...,...,...,...
6918,6919,Volvo,XC90,2015,Diesel,Semi-Automatic Gear,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription
6919,6920,Volvo,XC90,2017,Diesel,Automatic transmission,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription
6920,6921,Volvo,XC90,2018,Diesel,Automatic transmission,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription
6921,6922,Volvo,XC90,2019,Diesel,Automatic transmission,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription


In [61]:
model_Brand_Df = pd.DataFrame()
model_Brand_Df["brand"] = labelDf["BRAND"]
model_Brand_Df["model"] = labelDf["BRAND"]+ " "+ labelDf["MODEL"]

In [62]:
labelDf["CASETYPE"].unique()


array(['Sedan', 'Hatchback 3 Doors', 'Hatchback 5 Doors', 'Cabriolet',
       'Station Wagon', 'Other', 'Sport / Coupe', 'Off Road', 'Roadster',
       'MPV', 'Crossover', 'Minivan'], dtype=object)

In [63]:
model_Brand_Df["models_without_brand"] = model_Brand_Df['model'].apply(lambda x: ' '.join(x.split()[1:]))

In [64]:
model_Brand_Df.rename(columns = {'model':'models'}, inplace = True) 
model_Brand_Df.rename(columns = {'brand':'brands'}, inplace = True) 

In [65]:
model_Brand_Df

,brands,models,models_without_brand
0,Audi,Audi 100,100
1,Audi,Audi 100,100
2,Audi,Audi 80,80
3,Audi,Audi 80,80
4,Audi,Audi A1,A1
...,...,...,...
6918,Volvo,Volvo XC90,XC90
6919,Volvo,Volvo XC90,XC90
6920,Volvo,Volvo XC90,XC90
6921,Volvo,Volvo XC90,XC90


In [66]:
#detecting duplicates
model_Brand_Df.duplicated(subset= ['models']).sum()

6549

In [67]:
if (model_Brand_Df.duplicated(subset= ['models']).sum() != 0):
    model_Brand_Df.drop_duplicates(subset= ['models'],keep= 'first', inplace= True)


In [68]:
model_Brand_Df.duplicated(subset= ['models']).sum()

0

In [69]:
len(model_Brand_Df["brands"])

374

In [70]:
scraped_df = pd.read_csv(r"C:/Users/ASUS/Desktop/stage pfe/collected data/car_brands_models.csv")

In [71]:
scraped_df

,models,brands,models_without_brand
0,Abarth 500,Abarth,500
1,Abarth 600e,Abarth,600e
2,Abarth 124,Abarth,124
3,Abarth Grande Punto,Abarth,Grande Punto
4,Abarth Punto Evo,Abarth,Punto Evo
...,...,...,...
3321,Xinyang Electric Utility,Xinyang,Electric Utility
3322,Zagato Milano,Zagato,Milano
3323,Zastava 1100,Zastava,1100
3324,Zastava 1300,Zastava,1300


In [72]:
# Merging the two DataFrames
data = pd.concat([scraped_df, model_Brand_Df], ignore_index=True)

# Display the merged DataFrame
print(data)

                   models  brands models_without_brand
0              Abarth 500  Abarth                  500
1             Abarth 600e  Abarth                 600e
2              Abarth 124  Abarth                  124
3     Abarth Grande Punto  Abarth         Grande Punto
4        Abarth Punto Evo  Abarth            Punto Evo
...                   ...     ...                  ...
3695            Volvo V60   Volvo                  V60
3696            Volvo V70   Volvo                  V70
3697             Volvo XC   Volvo                   XC
3698           Volvo XC60   Volvo                 XC60
3699           Volvo XC90   Volvo                 XC90

[3700 rows x 3 columns]


In [73]:
#detecting duplicates
data.duplicated(subset= ['models']).sum()

228

In [74]:
if (data.duplicated(subset= ['models']).sum() != 0):
    data.drop_duplicates(subset= ['models'],keep= 'first', inplace= True)


In [75]:
'''
for i in range(len(df)):
    for word in df["Keyword"][i].split(" "):
        if word == "4x4":
            df["Keyword"][i].replace("4x4", "").strip()
'''


'\nfor i in range(len(df)):\n    for word in df["Keyword"][i].split(" "):\n        if word == "4x4":\n            df["Keyword"][i].replace("4x4", "").strip()\n'

In [76]:
df["Keyword"]

0       maison volkswagen tunisie pièce de rechange
1            prix pièce de rechange kia rio tunisie
2                   accessoires isuzu d max tunisie
3              pièce de rechange volkswagen tunisie
4                 pièce de rechange volkswagen sfax
                           ...                     
3207                            c3 aircross tunisie
3208                   prix peugeot 2008 en tunisie
3209                    citroen c3 aircross tunisie
3210                 prix hyundai tucson en tunisie
3211                      peugeot 3008 tunisie prix
Name: Keyword, Length: 3212, dtype: object

In [77]:
#suppression des accents qui existe dans les caractères accentués
df["Keyword"] = df["Keyword"].apply(unidecode)
df.head()

,Keyword,prediction
0,maison volkswagen tunisie piece de rechange,non générique
1,prix piece de rechange kia rio tunisie,non générique
2,accessoires isuzu d max tunisie,non générique
3,piece de rechange volkswagen tunisie,non générique
4,piece de rechange volkswagen sfax,non générique


**STEP 2: LABELING**
 

In [78]:
brands = list(data["brands"].unique()) 

In [79]:
models_without_brand = []
models_without_brand = data["models_without_brand"]

In [80]:
models_without_brand = list(models_without_brand)

In [81]:
len(models_without_brand)

3472

In [82]:
len(brands)

304

In [83]:
for i in range(len(brands)):
    brands[i] = brands[i].lower()

In [84]:
for i in range(len(models_without_brand)):
    models_without_brand[i] = models_without_brand[i].lower()

In [85]:
from keras.preprocessing.text import text_to_word_sequence
dataset = pd.DataFrame()
dataset['Keyword'] = df['Keyword']
# Apply text_to_word_sequence function to each keyword and store the tokens in the Tokenized_Keyword column
dataset['Tokenized_Keyword'] = df['Keyword'].apply(lambda x: text_to_word_sequence(x))

In [86]:
dataset

,Keyword,Tokenized_Keyword
0,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, de, recha..."
1,prix piece de rechange kia rio tunisie,"[prix, piece, de, rechange, kia, rio, tunisie]"
2,accessoires isuzu d max tunisie,"[accessoires, isuzu, d, max, tunisie]"
3,piece de rechange volkswagen tunisie,"[piece, de, rechange, volkswagen, tunisie]"
4,piece de rechange volkswagen sfax,"[piece, de, rechange, volkswagen, sfax]"
...,...,...
3207,c3 aircross tunisie,"[c3, aircross, tunisie]"
3208,prix peugeot 2008 en tunisie,"[prix, peugeot, 2008, en, tunisie]"
3209,citroen c3 aircross tunisie,"[citroen, c3, aircross, tunisie]"
3210,prix hyundai tucson en tunisie,"[prix, hyundai, tucson, en, tunisie]"


In [87]:
'''
def label_keyword(row):
    #tokens = [token.lower() for token in row['Tokenized_Keyword']]  # Convert tokens to lowercase for case-insensitive comparison
    #has_brand = any(brand.lower() in tokens for brand in brands)
    
    
    # For model checking, consider combinations of adjacent tokens
    #token_combinations = list(tokens)
    #for i in range(len(tokens) - 1):
    #    token_combinations.append(tokens[i] + ' ' + tokens[i + 1])
    
    #has_model = any(model.lower() in token_combinations for model in models_without_brand)
    
    for key in range(len(dataset)):
        for brand in brands

    if (has_brand and has_model) or has_model:
        return 'modèle'
    elif has_brand:
        return 'marque'
    else:
        return 'unknown'

# Reapply the labeling function
dataset['Label'] = dataset.apply(label_keyword, axis=1)

# Check the updated counts
label_counts = dataset['Label'].value_counts()
print(label_counts)

'''


"\ndef label_keyword(row):\n    #tokens = [token.lower() for token in row['Tokenized_Keyword']]  # Convert tokens to lowercase for case-insensitive comparison\n    #has_brand = any(brand.lower() in tokens for brand in brands)\n    \n    \n    # For model checking, consider combinations of adjacent tokens\n    #token_combinations = list(tokens)\n    #for i in range(len(tokens) - 1):\n    #    token_combinations.append(tokens[i] + ' ' + tokens[i + 1])\n    \n    #has_model = any(model.lower() in token_combinations for model in models_without_brand)\n    \n    for key in range(len(dataset)):\n        for brand in brands\n\n    if (has_brand and has_model) or has_model:\n        return 'modèle'\n    elif has_brand:\n        return 'marque'\n    else:\n        return 'unknown'\n\n# Reapply the labeling function\ndataset['Label'] = dataset.apply(label_keyword, axis=1)\n\n# Check the updated counts\nlabel_counts = dataset['Label'].value_counts()\nprint(label_counts)\n\n"

In [88]:
def has_brand(word,brands_list):
    for brand in brands_list:
        if word.lower() == brand:
            return True
        
    return False
    
def has_model(word,models_without_brand_list):
    for model in models_without_brand_list:
        if word.lower() == model:
            return True
    return False


fr_stops = set(stopwords.words('french'))

def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in fr_stops]


dataset["Label"] = ''
def label_keyword(dataset,brands_list,models_without_brand_list):    
    for i in range(len(dataset)):
        label = "unknown"
        for j in range(len(dataset.loc[i][1])):
            word = dataset.loc[i][1][j]
            print(word)
            if (has_brand(word,brands) == True):
                label = "marque"
                print("test1")
            if  (((has_brand(word,brands) == True) and (has_model(word,models_without_brand) == True)) or(has_model(word,models_without_brand) == True)):
                label = "modèle"
                print("test2")
            

        dataset["Label"][i] = label            
        print(dataset["Label"][i])    


In [89]:
# Remove stopwords from tokenized keywords
dataset.loc[:, 'Tokenized_Keyword'] = dataset.loc[:, 'Tokenized_Keyword'].apply(remove_stopwords)

In [90]:
dataset

,Keyword,Tokenized_Keyword,Label
0,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, rechange]",
1,prix piece de rechange kia rio tunisie,"[prix, piece, rechange, kia, rio, tunisie]",
2,accessoires isuzu d max tunisie,"[accessoires, isuzu, max, tunisie]",
3,piece de rechange volkswagen tunisie,"[piece, rechange, volkswagen, tunisie]",
4,piece de rechange volkswagen sfax,"[piece, rechange, volkswagen, sfax]",
...,...,...,...
3207,c3 aircross tunisie,"[c3, aircross, tunisie]",
3208,prix peugeot 2008 en tunisie,"[prix, peugeot, 2008, tunisie]",
3209,citroen c3 aircross tunisie,"[citroen, c3, aircross, tunisie]",
3210,prix hyundai tucson en tunisie,"[prix, hyundai, tucson, tunisie]",


In [91]:
label_keyword(dataset,brands, models_without_brand)

maison
volkswagen
test1
tunisie
piece
rechange
marque
prix
piece
rechange
kia
test1
rio
test2
tunisie
modèle
accessoires
isuzu
test1
max
tunisie
marque
piece
rechange
volkswagen
test1
tunisie
marque
piece
rechange
volkswagen
test1
sfax
marque
piece
rechange
hyundai
test1
i10
test2
tunisie
modèle
retroviseur
ford
test1
test2
fiesta
test2
tunisie
prix
modèle
prix
retroviseur
tunisie
unknown
accessoire
mercedes
test1
tunisie
marque
prix
piece
rechange
seat
test1
ibiza
test2
tunisie
modèle
accessoire
toyota
test1
hilux
test2
tunisie
modèle
piece
rechange
citroen
test1
ax
test2
tunisie
modèle
piece
rechange
hyundai
test1
i20
test2
tunisie
modèle
prix
retroviseur
symbol
test2
tunisie
modèle
mazda
test1
tunisie
piece
rechange
marque
accessoires
opel
test1
astra
test2
h
tunisie
modèle
piece
rechange
kia
test1
rio
test2
tunisie
modèle
piece
rechange
audi
test1
a4
test2
tunisie
modèle
pieces
rechange
volkswagen
test1
tunisie
marque
prix
kit
chaine
ford
test1
test2
fiesta
test2
tunisie
modèle
pri

location
batterie
zoe
test2
occasion
modèle
prix
dacia
test1
spring
test2
electrique
modèle
prix
recharge
zoe
test2
modèle
mercedes
test1
electrique
2021
prix
marque
prix
kwh
superchargeur
tesla
test1
marque
borne
voiture
electrique
prix
unknown
prix
fiat
test1
500
test2
electrique
3
test2
1
test2
modèle
500
test2
electrique
prix
modèle
voiture
electrique
zoe
test2
prix
modèle
borne
tesla
test1
maison
prix
marque
cout
location
batterie
zoe
test2
modèle
prix
hyundai
test1
kona
test2
electrique
64
kwh
204
test2
ch
executive
test2
modèle
ds
test1
test2
electrique
prix
modèle
ami
test2
electrique
prix
modèle
prix
renault
test1
zoe
test2
neuve
modèle
prix
borne
recharge
unknown
renault
test1
4l
electrique
prix
marque
tarif
borne
electrique
unknown
borne
recharge
electrique
prix
unknown
fiat
test1
500
test2
electrique
2020
prix
modèle
prix
kwh
borne
recharge
unknown
renault
test1
megane
test2
electrique
2022
prix
modèle
voiture
electrique
renault
test1
prix
marque
cupra
test1
born
test2
elec

In [92]:
dataset['Label'].value_counts()

Label
modèle     1976
marque      773
unknown     463
Name: count, dtype: int64

In [93]:
#dataset.drop("Label_Test",axis=1)

In [94]:
dataset[dataset['Label'] == 'modèle'].to_csv('C:/Users/ASUS/Desktop/stage pfe/collected data/modele2.csv', index=False)
dataset[dataset['Label'] == 'unknown'].to_csv('C:/Users/ASUS/Desktop/stage pfe/collected data/unknown2.csv', index=False)
dataset[dataset['Label'] == 'marque'].to_csv('C:/Users/ASUS/Desktop/stage pfe/collected data/marque2.csv', index=False)


In [95]:
unknown_data = pd.read_csv(r'C:/Users/ASUS/Desktop/stage pfe/collected data/unknown2.csv')
unknown_data

,Keyword,Tokenized_Keyword,Label
0,prix retroviseur tunisie,"['prix', 'retroviseur', 'tunisie']",unknown
1,pieces de rechanges tunisie,"['pieces', 'rechanges', 'tunisie']",unknown
2,borne de recharge voiture electrique prix,"['borne', 'recharge', 'voiture', 'electrique',...",unknown
3,prix installation borne de recharge,"['prix', 'installation', 'borne', 'recharge']",unknown
4,prix borne electrique,"['prix', 'borne', 'electrique']",unknown
...,...,...,...
458,lld sur vehicule d occasion,"['lld', 'vehicule', 'occasion']",unknown
459,loa avec reprise ancien vehicule,"['loa', 'reprise', 'ancien', 'vehicule']",unknown
460,loa ou lld occasion,"['loa', 'lld', 'occasion']",unknown
461,financement vehicule loa,"['financement', 'vehicule', 'loa']",unknown


In [96]:
unknown_data.head(20)

,Keyword,Tokenized_Keyword,Label
0,prix retroviseur tunisie,"['prix', 'retroviseur', 'tunisie']",unknown
1,pieces de rechanges tunisie,"['pieces', 'rechanges', 'tunisie']",unknown
2,borne de recharge voiture electrique prix,"['borne', 'recharge', 'voiture', 'electrique',...",unknown
3,prix installation borne de recharge,"['prix', 'installation', 'borne', 'recharge']",unknown
4,prix borne electrique,"['prix', 'borne', 'electrique']",unknown
5,elaris finn prix,"['elaris', 'finn', 'prix']",unknown
6,installation borne recharge voiture electrique...,"['installation', 'borne', 'recharge', 'voiture...",unknown
7,prix borne recharge voiture electrique,"['prix', 'borne', 'recharge', 'voiture', 'elec...",unknown
8,prix borne de recharge,"['prix', 'borne', 'recharge']",unknown
9,prix d un plein electrique,"['prix', 'plein', 'electrique']",unknown


**testing the model's prediction: "générique"**

In [97]:
dfG = pd.DataFrame()
# Filter rows where 'prediction' equals 'non générique'
filtered_rows_G = corpus[corpus["prediction"] == "générique"]

# Concatenate the filtered rows to df
dfG = pd.concat([dfG, filtered_rows_G], ignore_index=True)

In [98]:
dfG

,Keyword,prediction
0,accessoires auto tunisie,générique
1,accessoire voiture tunisie,générique
2,enjoliveur tunisie,générique
3,accessoire voiture tunisie prix,générique
4,accessoire voiture tunisie en ligne,générique
...,...,...
2539,location utilitaire tunis,générique
2540,circuit 4x4 tunisie marmara,générique
2541,excursion 4x4 hammamet,générique
2542,raid 4x4 en tunisie,générique


In [99]:
#suppression des accents qui existe dans les caractères accentués
dfG["Keyword"] = dfG["Keyword"].apply(unidecode)


#tokenization
# Apply text_to_word_sequence function to each keyword and store the tokens in the Tokenized_Keyword column
dfG['Tokenized_Keyword'] = dfG['Keyword'].apply(lambda x: text_to_word_sequence(x))

# Remove stopwords from tokenized keywords
dfG.loc[:, 'Tokenized_Keyword'] = dfG.loc[:, 'Tokenized_Keyword'].apply(remove_stopwords)



In [100]:
dfG.drop("prediction",axis= 1,inplace= True)

In [101]:
dfG

,Keyword,Tokenized_Keyword
0,accessoires auto tunisie,"[accessoires, auto, tunisie]"
1,accessoire voiture tunisie,"[accessoire, voiture, tunisie]"
2,enjoliveur tunisie,"[enjoliveur, tunisie]"
3,accessoire voiture tunisie prix,"[accessoire, voiture, tunisie, prix]"
4,accessoire voiture tunisie en ligne,"[accessoire, voiture, tunisie, ligne]"
...,...,...
2539,location utilitaire tunis,"[location, utilitaire, tunis]"
2540,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]"
2541,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]"
2542,raid 4x4 en tunisie,"[raid, 4x4, tunisie]"


In [102]:
dfG["Label"] = ""
label_keyword(dfG,brands, models_without_brand)

accessoires
auto
test1
tunisie
marque
accessoire
voiture
tunisie
unknown
enjoliveur
tunisie
unknown
accessoire
voiture
tunisie
prix
unknown
accessoire
voiture
tunisie
ligne
unknown
vente
accessoire
auto
test1
tunisie
gros
marque
vente
accessoire
voiture
tunisie
unknown
accessoire
interieur
voiture
tunisie
unknown
accessoire
voiture
gros
tunisie
unknown
accessoires
voiture
sfax
unknown
accessoire
auto
test1
tunisie
facebook
marque
courroie
alternateur
prix
tunisie
unknown
prix
pompe
direction
assistee
206
test2
tunisie
modèle
accessoire
megane
test2
2
test2
tunisie
modèle
piece
detachee
auto
test1
tunisie
marque
accessoire
voiture
sousse
unknown
prix
huile
boite
vitesse
test2
tunisie
modèle
accessoires
bmw
test1
tunisie
marque
accessoire
voiture
tunisie
unknown
piece
detachee
tunisie
unknown
piece
rechange
auto
test1
tunisie
marque
prix
piece
rechange
moto
tunisie
unknown
accessoires
auto
test1
tunis
marque
vente
accessoires
auto
test1
tunisie
marque
fournisseur
accessoire
auto
test1
tu

recharge
voiture
electrique
unknown
cout
recharge
voiture
electrique
unknown
voiture
electrique
moins
30000
eur
unknown
prix
recharge
electrique
voiture
unknown
voiture
electrique
occasion
moins
chere
unknown
cout
recharge
voiture
electrique
a
test2
maison
modèle
prix
recharge
voiture
electrique
unknown
cout
entretien
voiture
electrique
unknown
prix
recharge
voiture
electrique
unknown
petite
voiture
electrique
prix
unknown
voiture
electrique
moins
cher
unknown
cout
recharge
voiture
electrique
unknown
voiture
a
test2
hydrogene
prix
modèle
prix
recharge
voiture
electrique
autoroute
unknown
location
batterie
voiture
electrique
unknown
meilleur
rapport
qualite
prix
voiture
electrique
unknown
prix
recharge
voiture
electrique
leclerc
unknown
voiture
electrique
moins
chere
marche
unknown
voiture
electrique
neuve
moins
chere
unknown
combien
coute
recharge
voiture
electrique
unknown
porsche
test1
electrique
taycan
test2
prix
modèle
cout
recharge
electrique
unknown
tarif
recharge
voiture
electri

In [103]:
dfG['Label'].value_counts()

Label
unknown    2009
modèle      366
marque      169
Name: count, dtype: int64

In [104]:
dfG[dfG['Label'] == 'modèle'].to_csv('C:/Users/ASUS/Desktop/stage pfe/collected data/modele_test_grq.csv', index=False)
dfG[dfG['Label'] == 'unknown'].to_csv('C:/Users/ASUS/Desktop/stage pfe/collected data/unknown_test_grq.csv', index=False)
dfG[dfG['Label'] == 'marque'].to_csv('C:/Users/ASUS/Desktop/stage pfe/collected data/marque_test_grq.csv', index=False)

In [105]:
marque_gene = pd.read_csv(r'C:/Users/ASUS/Desktop/stage pfe/collected data/modele_test_grq.csv')
marque_gene

,Keyword,Tokenized_Keyword,Label
0,prix pompe direction assistee 206 tunisie,"['prix', 'pompe', 'direction', 'assistee', '20...",modèle
1,accessoire megane 2 tunisie,"['accessoire', 'megane', '2', 'tunisie']",modèle
2,prix huile boite de vitesse tunisie,"['prix', 'huile', 'boite', 'vitesse', 'tunisie']",modèle
3,cout recharge voiture electrique zoe,"['cout', 'recharge', 'voiture', 'electrique', ...",modèle
4,porsche taycan electrique prix,"['porsche', 'taycan', 'electrique', 'prix']",modèle
...,...,...,...
361,4x4 en tunisie,"['4x4', 'tunisie']",modèle
362,circuit 4x4 tunisie marmara,"['circuit', '4x4', 'tunisie', 'marmara']",modèle
363,excursion 4x4 hammamet,"['excursion', '4x4', 'hammamet']",modèle
364,raid 4x4 en tunisie,"['raid', '4x4', 'tunisie']",modèle


In [106]:
for i in range(len(models_without_brand)):
    if i == "20":
        print("found")
    
print("not found")  

#na7i 4x4 mel les modeles!!!!!

not found
